特征选择的目标就是：

找出对目标值（标签值）有影响的特征，去掉冗余和噪声信息。（噪声：部分特征对预测结果有负影响）这样，可以带来的好处是：

• 降低模型复杂度

• 提高模型的正化能力

• 提升模型的可解释性

---

如何进行特征选择！

过滤法
- 基于特征“统计属性”筛选（如方差、相关性），代表工具有 方差阈值（VarianceThreshold）

嵌入法
- 利用模型自身的特征重要性指标来进行筛选，比如：线性模型（Lasso、Ridge）、决策树、随机森林

包裹法
- 用模型“反馈”迭代筛选（如递归消除RFE）




## 过滤法

In [4]:
import numpy as np
from sklearn.feature_selection import VarianceThreshold

#创建特征数据
X=np.array([
    [0,0,1],
    [0,1,0],
    [1,0,0],
    [0,1,1],
    [1,0,1]])

# 计算特征的方差
variances = np.var(X, axis=0)
print("特征的方差:", variances)

# 选择方差大于0.1的特征
selector = VarianceThreshold(threshold=0.1)
X_selected = selector.fit_transform(X)
print("选择的特征:", X_selected)


特征的方差: [0.24 0.24 0.24]
选择的特征: [[0 0 1]
 [0 1 0]
 [1 0 0]
 [0 1 1]
 [1 0 1]]


## 嵌入法


``` py
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
#嵌入法
# 假设 X_train, y_train 已经准备好
# 使用随机森林作为基模型进行嵌入法特征选择
selector = SelectFromModel(
    RandomForestClassifier(n_estimators=100, random_state=42),
    threshold='median'  # 选择重要性中位数以上的特征
)
X_train_selected = selector.fit_transform(X_train, y_train)

# 获取被选中的特征索引
selected_idx = selector.get_support(indices=True)
print("选中的特征索引：", selected_idx)



## 包裹法

In [7]:
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
# 包裹法
def wrapper_feature_selection(X, y, k=10, estimator=None, cv=5):
    """
    包裹法特征选择：使用递归特征消除（RFE）或交叉验证选择最优特征子集
    
    参数:
        X: 特征矩阵，shape (n_samples, n_features)
        y: 目标向量，shape (n_samples,)
        k: 期望保留的特征数，若为'int'则直接保留top-k；若为'float'则按比例保留
        estimator: 用于评估特征重要性的模型，默认使用RandomForest
        cv: 交叉验证折数
    
    返回:
        selected_features: 被选中的特征索引列表
        support_mask: 布尔掩码，True表示该特征被选中
        feature_importance: 特征重要性排序（若estimator支持）
    """
    
    if estimator is None:
        estimator = RandomForestClassifier(n_estimators=100, random_state=42)
    
    # 若特征为连续值，先归一化以便基于树的模型处理
    if np.issubdtype(X.dtype, np.floating):
        scaler = MinMaxScaler()
        X_scaled = scaler.fit_transform(X)
    else:
        X_scaled = X
    
    # 使用RFECV进行包裹式特征选择
    selector = RFECV(estimator=estimator, step=1, cv=cv, scoring='accuracy', n_jobs=-1)
    selector.fit(X_scaled, y)
    
    # 获取结果
    selected_features = np.where(selector.support_)[0].tolist()
    support_mask = selector.support_
    if hasattr(selector.estimator_, 'feature_importances_'):
        feature_importance = selector.estimator_.feature_importances_
    else:
        feature_importance = None
    
    return selected_features, support_mask, feature_importance
